In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
import argparse

# ===================== 데이터셋 유틸리티 =====================
class TextDataset(Dataset):
    def __init__(self, token_ids, seq_len):
        self.token_ids = token_ids         # 전체 텍스트를 토큰화한 리스트
        self.seq_len = seq_len             # 모델 입력 시퀀스 길이 (ex. 32)

    def __len__(self):
        # 전체 데이터 길이에서 시퀀스 길이만큼 제외한 개수만큼 샘플을 만들 수 있음
        # ex. input: [10, 20, 30],[20, 30, 40], [30, 40, 50] target: [20, 30, 40], [30, 40, 50], [40, 50, 60]
        return len(self.token_ids) - self.seq_len

    def __getitem__(self, idx):
        # 입력: 현재 위치부터 seq_len까지 슬라이싱
        # 정답: 한 칸 오른 다음 토큰 시퀀스
        return (
            torch.tensor(self.token_ids[idx:idx + self.seq_len], dtype=torch.long),
            torch.tensor(self.token_ids[idx + 1:idx + self.seq_len + 1], dtype=torch.long)
        )


def build_dataloader(token_ids, seq_len, batch_size):
    dataset = TextDataset(token_ids, seq_len)  # 커스텀 Dataset 생성
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)  # DataLoader로 래핑해서 배치 단위로 학습 가능하게 함

In [ ]:
# ===================== 모델 구성 요소 =====================
class ScaledDotProductAttention(nn.Module):  # Q, K, V를 받아 attention 연산 수행
    def __init__(self):
        super().__init__()

    def forward(self, Q, K, V, attn_mask=None):
        # K.transpose: (B,H,T,D) -> (B,H,D,T), T: 시퀀스 길이, D: head당 임베딩 차원
        scores = torch.matmul(Q, K.transpose(-1, -2)) / (Q.size(-1) ** 0.5)  # QK^T / sqrt(d_k): 스케일 조정된 유사도
        if attn_mask is not None:  # 마스크가 있으면 future token 가리기
            scores = scores.masked_fill(attn_mask, -1e9)  # 마스크된 위치는 매우 작은 값으로 만들어 softmax에서 0 되게 함
        attn_weights = F.softmax(scores, dim=-1)  # attention weight 계산
        context = torch.matmul(attn_weights, V)  # attention weight와 V를 곱해 최종 컨텍스트 생성
        return context, attn_weights


class MultiHeadAttention(nn.Module):  # 멀티헤드 어텐션: 여러 개의 self-attention을 병렬 수행
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0  # head 수가 모델 차원 나눠떨어져야 함
        self.d_head = d_model // n_heads  # 각 head의 차원
        self.n_heads = n_heads

        self.W_Q = nn.Linear(d_model, d_model)  # 전체 Q, K, V에 대해 선형 변환
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        self.fc = nn.Linear(d_model, d_model)  # 모든 head 결과 concat 후 다시 선형 변환
        self.attention = ScaledDotProductAttention()

    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # Q, K, V 각각 (B, T, d_model) → (B, n_heads, T, d_head) projection 후 여러 head로 나눔
        # 여러 head로 나누는 이유는 모델의 표현력과 학습 안정성을 병렬로 학습하게 함
        # view만으로는 tensor의 물리적 위치 바꿀 수 x -> transpose 필수
        # view 이후 transpose는 안전함 (메모리 순서에서 안전하게 동작 후 transpose에서 새로 해석)
        Q = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)
        K = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)
        V = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)

        # 각 head에 대해 attention 계산
        context, attn_weights = self.attention(Q, K, V, attn_mask)
        # concat: (B, n_heads, T, d_head) → (B, T, d_model)
        # concat은 T기준: head들을 한 토큰 위치에 대해 이어붙이기 위함
        # transpose(비연속 메모리 만듦) -> contiguous(view 이전 필수) -> view(여기서 concat: (B,T,d_model))
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_head)
        output = self.fc(context)  # 최종 출력 projection
        return output, attn_weights


class PositionWiseFeedForwardNetwork(nn.Module):  # FFN: 각 위치별 독립적으로 2-layer MLP 수행
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)  # 확장
        self.fc2 = nn.Linear(d_ff, d_model)  # 다시 축소

    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))  # GELU 활성화 사용 (GPT-1 논문 따라)
         # smooth한 RELU + 약간의 확률성 -> 부드러운 곡선

class DecoderLayer(nn.Module):  # 하나의 디코더 층 = self-attn + FFN + residual + norm
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads)  # 멀티헤드 self-attention
        self.ffn = PositionWiseFeedForwardNetwork(d_model, d_ff)  # FFN
        self.norm1 = nn.LayerNorm(d_model)  # LayerNorm은 residual 뒤에 적용
        self.norm2 = nn.LayerNorm(d_model) # GPT-2에선 layernorm residual 앞으로 바뀜: Pre-LN 입력이 정규화돼서 학습 안정성 증가
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, attn_mask):
        attn_out, attn_weights = self.self_attn(x, x, x, attn_mask)  # 자기 자신에 대한 attention
        x = self.norm1(x + self.dropout1(attn_out))  # residual + norm
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.dropout2(ffn_out))  # residual + norm
        return x, attn_weights


class TransformerDecoder(nn.Module):  # 디코더 전체 = 여러 개의 DecoderLayer 쌓은 구조
    def __init__(self, n_layers, d_model, n_heads, d_ff, vocab_size, max_len, dropout):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)  # 토큰 임베딩 (B, T, d_model)로 바뀜
        self.pos_emb = nn.Embedding(max_len, d_model)       # 위치 임베딩 (정수 위치 기반), max_len: 입력 인덱스 범
        self.dropout = nn.Dropout(dropout)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)
        ])  # layer n개 쌓기

    def forward(self, inputs, attn_mask=None):
        # input.shape = (batch_size=4, seq_len=8),
        # inputs.size(1): 8, arange하면 1~7까지 각 위치 인덱스 나타내는 벡터
        # unsqueeze하면 [[]] 차원 하나 늘어남: (8) -> (1,8)
        # expand_as(inputs): shape(4,8)인 inputs과 동일한 크기로 broadcasting (1,8) -> (4,8) 즉, 각 배치에 같은 위치 인덱스 부여
        positions = torch.arange(inputs.size(1), device=inputs.device).unsqueeze(0).expand_as(inputs)  # 위치 인덱스 생성
        x = self.token_emb(inputs) + self.pos_emb(positions)  # 입력 임베딩 + 위치 임베딩
        x = self.dropout(x)

        attn_weights_list = []  # attention weight 기록용 (분석용)
        for layer in self.layers:
            x, attn_weights = layer(x, attn_mask)  # 디코더 레이어 순차 적용(forward callable)
            attn_weights_list.append(attn_weights)

       return x, attn_weights_list  # 최종 출력 + 각 layer의 attention weights


class GPT(nn.Module):  # GPT-1 본체: TransformerDecoder를 wrapping: 다른 모듈(TransformerDecoder)을 감싸 더 큰 모듈 만듦
    def __init__(self, decoder):
        super().__init__()
        self.decoder = decoder

    def forward(self, inputs, attn_mask=None):
        outputs, attn_weights = self.decoder(inputs, attn_mask) # callable
        return outputs, attn_weights


class GPTLMHead(nn.Module):  # GPT + 언어 모델링 헤드 (vocab 차원으로 projection)
    def __init__(self, gpt, vocab_size):
        super().__init__()
        self.gpt = gpt
        # (B, T, d_model) 형태의 hiddden state 출력 -> 여기에 vocab_sie만큼 linear projection
        # -> 어떤 단어일 확률(logits), 즉 [batch, seq_len, d_model] -> [batch, seq_len, vocab_size]
        self.lm_head = nn.Linear(gpt.decoder.token_emb.embedding_dim, vocab_size, bias=False)  # GPT 출력 → vocab logits
        # 토큰 임베딩과 weight tying할 수 있게 bias 제거한 형태(nn.Linear와 입력 임베딩 간의 weight)

    def forward(self, inputs, attn_mask=None):
        # gpt 본체 호출 -> [batch, seq_len, d_mocel] 출력(attention weights는 무시)
        outputs, _ = self.gpt(inputs, attn_mask)  # GPT 본체 통과
        # vocab 크기만큼 projection logits.shape = (batch_size, seq_len, vocab_size)
        logits = self.lm_head(outputs)  # 각 위치별 다음 토큰 확률 분포 출력
        return logits


# ===================== 어텐션 마스크 유틸리티 =====================
def create_causal_mask(seq_len, device):
    mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1).bool()  # 상삼각 행렬 생성 (미래 가리기)
    # diagonal=1: 대각선 기준 위쪽으로 1만 남기고 아래쪽 0 -> bool로 변환하면 True, False
    # True는 -1e9로 만들어서 softmax에서 0이 되게함
    return mask.unsqueeze(0).unsqueeze(1)  # (1, 1, seq_len, seq_len): 배치, 헤드 차원 맞춤

# Causal Mask가 Attention Score에 적용되는 예시

GPT는 **자기보다 미래 토큰을 보면 안 되므로**,  
**Causal Mask**를 사용해 Attention Score에서 **미래 위치를 가린다**.  
이때 사용하는 마스크는 **상삼각 행렬(triu)**이다.

---

## 목표
각 토큰이 **자기 위치 이전/현재까지만 attend**하고,  
**미래 위치는 softmax에서 0 확률로 강제**하는 것.

---

## 예시: 시퀀스 길이 T = 4 (batch=1, head=1)

### 원래 Attention Score (`QKᵀ / sqrt(d_k)` 결과)

|         | 토큰 0 | 토큰 1 | 토큰 2 | 토큰 3 |
|---------|--------|--------|--------|--------|
| 토큰 0  |  0.1   |  0.2   |  0.3   |  0.4   |
| 토큰 1  |  0.5   |  0.6   |  0.7   |  0.8   |
| 토큰 2  |  0.9   |  1.0   |  1.1   |  1.2   |
| 토큰 3  |  1.3   |  1.4   |  1.5   |  1.6   |

---

### Causal Mask (`torch.triu(torch.ones(T, T), diagonal=1)`)
[[False, True, True, True ],

[False, False, True, True ],

[False, False, False, True ],

[False, False, False, False]]

### 마스크 적용 후 (`masked_fill(attn_mask, -1e9)`)

|         | 토큰 0 | 토큰 1 | 토큰 2 | 토큰 3 |
|---------|--------|--------|--------|--------|
| 토큰 0  |  0.1   | -1e9   | -1e9   | -1e9   |
| 토큰 1  |  0.5   |  0.6   | -1e9   | -1e9   |
| 토큰 2  |  0.9   |  1.0   |  1.1   | -1e9   |
| 토큰 3  |  1.3   |  1.4   |  1.5   |  1.6   |

---

### softmax 적용 결과 (예: 토큰 2의 row)

softmax([0.9, 1.0, 1.1, -1e9]) ≈ [0.3006, 0.3322, 0.3671, 0.0000]

토큰 2는 미래 토큰(토큰 3)을 절대 보지 않음

# GPT 차원 흐름 정리

아래는 GPT 구조 (Decoder-only Transformer)에서 입력부터 출력까지의 **텐서 차원 변화**를 정리한 표입니다.  
입력은 `[B, T]`이고, 최종 출력은 `[B, T, V]`입니다.  
(GPTLMHead까지 포함)

---

## 기본 설정

- `B` = Batch size  
- `T` = Sequence length  
- `D` = d_model (예: 256)  
- `H` = n_heads (예: 4)  
- `d_head` = D // H (예: 64)  
- `V` = vocab_size (예: 50257)

---

## 전체 차원 흐름

| 단계 | 연산 | 입력 차원 | 출력 차원 | 설명 |
|------|------|-----------|-----------|------|
| 1 | **Input IDs** | `[B, T]` | - | 정수 토큰 ID |
| 2 | `token_emb(inputs)` | `[B, T]` | `[B, T, D]` | 토큰 임베딩 |
| 3 | `pos_emb(positions)` | `[B, T]` | `[B, T, D]` | 위치 임베딩 |
| 4 | `x = token_emb + pos_emb` | `[B, T, D]` | `[B, T, D]` | 입력 인코딩 |
| 5 | `DecoderLayer` 반복 | `[B, T, D]` | `[B, T, D]` | layer n번 반복 (shape 유지) |
| 6 | **MultiHeadAttention 내부** |
| 6-1 | `Q = W_Q(x)` | `[B, T, D]` | `[B, T, D]` | 선형 변환 |
| 6-2 | `Q.view().transpose()` | `[B, T, D]` | `[B, H, T, d_head]` | head로 분리 |
| 6-3 | Attention 결과 | `[B, H, T, d_head]` | `[B, H, T, d_head]` | context 벡터 |
| 6-4 | concat & fc | `[B, H, T, d_head]` | `[B, T, D]` | 모든 head 이어붙임 |
| 7 | **Feed Forward Network** | `[B, T, D]` → `[B, T, d_ff]` → `[B, T, D]` | `[B, T, D]` | 2-layer MLP |
| 8 | Residual + LayerNorm | `[B, T, D]` | `[B, T, D]` | 안정성 증가 |
| 9 | **Decoder 출력** | `[B, T, D]` | `[B, T, D]` | Transformer 전체 출력 |
| 10 | `lm_head(x)` | `[B, T, D]` | `[B, T, V]` | vocab projection (logits) |
| 11 | CrossEntropyLoss | `[B, T, V]` vs `[B, T]` | `scalar` | 학습 손실 계산 |

---

## 최종 정리

```text
입력          : [B, T]         → 정수 토큰 ID
↓ 임베딩
임베딩        : [B, T, D]
↓ 디코더 n층
디코더 출력   : [B, T, D]
↓ LM head (vocab projection)
로짓          : [B, T, V]
↓ Loss 계산
손실          : scalar


In [ ]:
# ===================== 학습 루프 =====================
class Trainer:
    def __init__(self, model, vocab_size, device, args):
        self.model = model                                # GPTLMHead 모델
        self.vocab_size = vocab_size                      # CrossEntropyLoss에 필요한 vocab 크기
        self.device = device                              # 'cuda' or 'cpu'
        self.args = args                                  # 하이퍼파라미터 설정 객체
        self.criterion = nn.CrossEntropyLoss()            # 다음 토큰 예측이므로 CrossEntropy 사용
        self.optimizer = optim.Adam(self.model.parameters(), lr=args.lr)  # GPT 논문에서도 Adam 사용

    def pretrain(self, dataloader):                       # 학습 루프
        self.model.train()                                # 학습 모드 켜기
        total_loss = 0
        for batch in dataloader:                          # 배치 단위 반복
            inputs = batch[0].to(self.device)             # input: (B, T)
            targets = batch[1].to(self.device)            # target: (B, T)
            attn_mask = create_causal_mask(inputs.size(1), inputs.device)  # future token 가리는 causal mask

            logits = self.model(inputs, attn_mask=attn_mask)  # (B, T, vocab)
            loss = self.criterion(                           # flatten해서 CrossEntropyLoss 계산
                logits.view(-1, self.vocab_size),            # (B*T, vocab)
                targets.view(-1)                             # (B*T)
            )

            self.optimizer.zero_grad()  # 기울기 초기화
            loss.backward()             # 역전파
            self.optimizer.step()       # 파라미터 업데이트

            total_loss += loss.item()   # loss 누적

        return total_loss / len(dataloader)  # 평균 loss 반환

    def evaluate(self, dataloader):                    # 평가 루프 (gradient 끄고)
        self.model.eval()                              # 평가 모드 (dropout, layer norm 등 다르게 동작)
        total_loss = 0
        with torch.no_grad():                          # autograd off
            for batch in dataloader:
                inputs = batch[0].to(self.device)
                targets = batch[1].to(self.device)
                logits = self.model(inputs)            # causal mask 생략 (inference에서도 써주는 게 낫지만 여기선 생략)
                loss = self.criterion(
                    logits.view(-1, self.vocab_size),
                    targets.view(-1)
                )
                total_loss += loss.item()
        return total_loss / len(dataloader)            # 평균 loss 반환

In [ ]:
# ===================== 실행 코드 =====================
def main():
    class Args:  # 하이퍼파라미터 설정
        mode = 'pretrain'         # 현재 코드는 pretrain 모드만 존재
        seq_len = 32              # 입력 시퀀스 길이 (GPT 논문은 최대 512)
        batch_size = 4            # 배치 크기
        n_layer = 4               # Transformer decoder layer 개수 (GPT-1은 12개)
        d_model = 256             # hidden size (GPT-1은 768)
        n_heads = 4               # multi-head attention head 수 (GPT-1은 12)
        d_ff = 1024               # FFN 내부 차원 (GPT-1은 3072)
        dropout = 0.1             # dropout 비율
        lr = 3e-4                 # learning rate
        epochs = 10               # 학습 epoch 수
    args = Args()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용 가능하면 CUDA

    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")   # HuggingFace에서 GPT-2 tokenizer 사용
    tokenizer.pad_token = tokenizer.eos_token           # GPT-2는 padding 토큰이 없어서 eos로 대체
    vocab_size = tokenizer.vocab_size                   # vocabulary 크기 (50257)

    # 입력 텍스트 인코딩 (GPT-1은 대용량 코퍼스 사용하지만 여기선 예제 텍스트)
    text = (
    "GPT models are trained to predict the next word. "
    "They can be applied to various NLP tasks such as translation, summarization, and question answering. "
    "This model is an implementation of GPT-1 using PyTorch. "
    "The model uses a Transformer decoder stack to model sequences."
    )
    token_ids = tokenizer.encode(text)  # 텍스트 → 토큰 ID (1D 리스트)

    # 데이터로더 구성: 슬라이딩 윈도우 방식으로 input/target 쌍 만들고 배치 단위로 묶음
    dataloader = build_dataloader(token_ids, args.seq_len, args.batch_size)

    # 모델 초기화
    decoder = TransformerDecoder(                 # GPT-1은 decoder-only 구조
        n_layers=args.n_layer,                    # 디코더 layer 수
        d_model=args.d_model,                     # hidden dim
        n_heads=args.n_heads,                     # attention head 수
        d_ff=args.d_ff,                           # FFN 차원
        vocab_size=vocab_size,                    # vocab size for token embedding
        max_len=args.seq_len,                     # position embedding 길이
        dropout=args.dropout                      # dropout
    )
    model = GPTLMHead(GPT(decoder), vocab_size).to(device)  # GPT 본체 + LM head 결합

    # 트레이너 생성 후 학습 시작
    trainer = Trainer(model, vocab_size, device, args)
    for epoch in range(args.epochs):
        loss = trainer.pretrain(dataloader)       # 한 epoch 학습
        print(f"[Epoch {epoch+1}] Loss: {loss:.4f}")  # 평균 loss 출력

    # 평가 및 결과 출력
    val_loss = trainer.evaluate(dataloader)
    print(f"최종 평가 손실: {val_loss:.4f}")

    return model, tokenizer, device  # 학습된 모델과 tokenizer 반환

# main 실행 → 학습 진행
model, tokenizer, device = main()

[Epoch 1] Loss: 9.6531
[Epoch 2] Loss: 7.1597
[Epoch 3] Loss: 5.1862
[Epoch 4] Loss: 3.6417
[Epoch 5] Loss: 2.5021
[Epoch 6] Loss: 1.7133
[Epoch 7] Loss: 1.1560
[Epoch 8] Loss: 0.7799
[Epoch 9] Loss: 0.5300
[Epoch 10] Loss: 0.3674
최종 평가 손실: 0.4821


In [ ]:
def generate_next_token(model, tokenizer, prompt, max_new_tokens=20, device='cpu'):
    model.eval()  # 모델을 평가 모드로 전환 (Dropout, LayerNorm 등의 동작 달라짐)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)  # 프롬프트 문자열을 토큰으로 변환 후 tensor로 만듦

    with torch.no_grad():  # 생성 시 gradient 계산 불필요 → 메모리/속도 최적화
        for _ in range(max_new_tokens):  # 지정된 길이만큼 반복 생성
            # causal mask 생성: 미래 토큰을 참고하지 못하도록 상삼각 행렬 형태로 마스킹
            attn_mask = torch.triu(
                torch.ones((1, input_ids.size(1), input_ids.size(1)), device=device),
                diagonal=1
            ).bool().unsqueeze(1)  # (1, 1, T, T)

            logits = model(input_ids, attn_mask=attn_mask)  # 현재까지 입력에 대한 예측 logits 얻기
            next_token_logits = logits[:, -1, :]            # 마지막 위치의 토큰에 대한 출력만 사용
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)  # 가장 확률 높은 토큰 선택

            input_ids = torch.cat([input_ids, next_token], dim=1)  # 생성된 토큰을 다음 입력으로 이어붙임

    output_text = tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)  # 토큰을 다시 문자열로 디코딩
    return output_text  # 최종 생성 텍스트 반환

In [ ]:
# 학습 완료 후 호출
prompt = "GPT is"
generated = generate_next_token(model, tokenizer, prompt, max_new_tokens=30, device=device)
print("=== 생성 결과 ===")
print(generated)

=== 생성 결과 ===
GPT is an implementation of GPT-1 using PyTorch. The model uses a Transformer decoder stack to various NLP tasks such as translation,


1. 생성 결과가 학습 텍스트와 얼마나 겹치는가?

    text = (
        "GPT models are trained to predict the next word. "
        "They can be applied to various NLP tasks such as translation, summarization, and question answering. "
        "This model is an implementation of GPT-1 using PyTorch. "
        "The model uses a Transformer decoder stack to model sequences."
    )

    → 즉, 모델은 다음과 같은 문장을 학습함:

        This model is an implementation of GPT-1 using PyTorch.
        The model uses a Transformer decoder stack to model sequences.
        ... NLP tasks such as translation ...

    → 생성된 문장과 거의 일치함.

2. 이건“생성”인가, 아니면 “복사”인가?

    현재 결과는 **기억 기반 복원(memorization)**에 가까움.
    training data에서 거의 동일한 문장이 있었고,prompt도 "GPT is"로 정확히 그 문장의 시작과 일치했기 때문에 모델은 학습된 문장을 거의 그대로 “회상”한 것에 가까움.

3. GPT-1처럼 작동하고 있긴 한가?

    GPT-1은 언어 모델로서, 다음 토큰을 예측하는 self-supervised 학습을 통해 autoregressive 방식으로 텍스트 생성을 수행함.
    현재 모델도 GPT 구조를 따르고, next token prediction을 기반으로 학습되었고, generate_next_token()에서 토큰을 하나씩 생성하며 문장을 예측하고 있음.
    따라서 구조적으론 GPT-1이 맞고, 동작도 일관적임.

4. "창의적인” 생성은 어떻게 유도할 수 있을까?

    현재 학습 데이터가 짧고, 모델도 작은 편이어서 모델이 외운 것을 꺼내는 수준임.

    개선 방향:
    더 긴 텍스트와 다양한 문장으로 pretrain
    top_k, top_p sampling 같은 sampling 전략 추가 (지금은 argmax)
    학습 epoch 증가 + weight decay 조절 → 일반화 향상
    학습 텍스트와 다른 prompt 제공 → 예: "Artificial intelligence is"